In [1]:
# Importamos la Libreria de Pandas (Manejo de DataFrames)
import pandas as pd
# Importamos la libreria de Numpy para las operaciones numéricas
import numpy as np
# importamos el modelo RandomForestRefressor  
from sklearn.ensemble import RandomForestRegressor
# importamos métricas para calcular el error absoluto
from sklearn.metrics import mean_absolute_error
# Importamos el modelo de seleccion de datos para dividir
from sklearn.model_selection import train_test_split
# Importamos Imputer para el manejo de valores Faltantes
from sklearn.impute import SimpleImputer
# Importamos el modelo de machine learning: DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
# Importamos el modelo de machine learning: XGBRegressor
from xgboost import XGBRegressor
# Importamos el modelo de machine learning: SVM
from sklearn.svm import SVR
#from sklearn import svm
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

## 1. Importacion de set de datos

In [2]:
# Diccionario de datos
df_dictionary = pd.read_csv('Dictionary.csv')
# Modelos de estructura de datos para enviar la solución
df_samplesubmission = pd.read_csv('samplesubmission.csv')
# Identificadores de los item a ser probados
df_solution_template = pd.read_csv('solution_template.csv')
# Datos de entrenamiento
df_training_v2 = pd.read_csv('training_v2.csv')
# Datos de prueba
df_test_unlabeled = pd.read_csv('unlabeled.csv')

In [3]:
# Vista del Diccionario de datos
display( df_dictionary.head() )
print("\nTamano del diccionario de datos")
df_dictionary.shape

,Category,Variable Name,Unit of Measure,Data Type,Description,Example
0,identifier,encounter_id,None,integer,Unique identifier associated with a patient un...,None
1,identifier,hospital_id,None,integer,Unique identifier associated with a hospital,None
2,identifier,patient_id,None,integer,Unique identifier associated with a patient,None
3,demographic,hospital_death,None,binary,Whether the patient died during this hospitali...,0
4,demographic,age,Years,numeric,The age of the patient on unit admission,None



Tamano del diccionario de datos


(188, 6)

In [4]:
# Vista Modelo de estructura de datos para enviar la solución
display( df_samplesubmission.head() )
print("\nTamano del modelo de estructura")
df_samplesubmission.shape
# De ahora en adelante lo descartamos

,encounter_id,hospital_death
0,2,0.500
1,5,0.200
2,7,0.001



Tamano del modelo de estructura


(3, 2)

In [5]:
# Identificadores de los item a ser probados
display( df_solution_template.head() )
print("\nTamano de los items")
df_solution_template.shape

,encounter_id,hospital_death
0,2,NaN
1,5,NaN
2,7,NaN
3,8,NaN
4,10,NaN



Tamano de los items


(39308, 2)

In [6]:
# Vista previa de Datos de entrenamiento
display( df_training_v2.head() )
print("\nTamano de los Datos de entrenamiento")
display( df_training_v2.shape )

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,...,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0.0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,46.0,37.0,37.0,51.0,7.45,...,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122.0,703.03,0,0.0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0.0,68.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203.0,1206.03,1,0.0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1.0,60.0,30.0,30.0,142.0,7.39,...,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0.04,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119.0,601.01,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0.0,103.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma



Tamano de los Datos de entrenamiento


(91713, 186)

In [7]:
# Vista previa de Datos de prueba
display( df_test_unlabeled.head() )
print("\nTamano de los Datos de prueba")
df_test_unlabeled.shape

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,...,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,2,127112,7,NaN,56.0,21.102439,0,Caucasian,M,180.3,Emergency Department,Accident & Emergency,1105,admit,Neuro ICU,0.104167,0,68.6,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
1,5,130737,178,NaN,NaN,19.500595,1,Caucasian,F,145.0,Operating Room,Operating Room / Recovery,1043,admit,Med-Surg ICU,0.230556,0,41.0,NaN,217.0,1502.02,1,0.0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,60.0,NaN,0.0,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
2,7,101458,86,NaN,66.0,32.518596,1,African American,M,170.2,Operating Room,Operating Room / Recovery,1035,admit,CCU-CTICU,0.140972,0,94.2,NaN,302.0,1208.06,1,0.0,NaN,22.0,1.84,NaN,2.0,5.0,0.0,1.0,81.0,124.0,39.2,0.0,115.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
3,8,48067,163,NaN,64.0,20.798520,0,Caucasian,M,182.8,Operating Room,Operating Room / Recovery,1078,admit,Med-Surg ICU,0.002083,0,69.5,1.9,303.0,211.03,0,0.0,0.8,24.0,0.75,0.7,4.0,6.0,0.0,1.0,122.0,111.0,32.0,1.0,48.0,36.0,36.0,142.0,7.35,...,1.4,1.4,NaN,NaN,248.0,248.0,4.0,4.0,133.0,133.0,9.7,9.7,36.0,36.0,7.37,7.35,142.0,57.0,202.857143,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.19,0.17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Respiratory,Respiratory
4,10,25865,59,NaN,56.0,NaN,0,Caucasian,M,NaN,Emergency Department,Accident & Emergency,1012,admit,Neuro ICU,0.004861,0,66.4,NaN,217.0,1501.01,1,0.0,NaN,11.0,0.52,NaN,1.0,4.0,0.0,1.0,192.0,94.0,32.8,0.0,41.0,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic



Tamano de los Datos de prueba


(39308, 186)

## 2. Exploration de tipos de columnas, informacion y describe

In [8]:
display( df_solution_template.info() )
print("**Los items de la solucion 2  (encounter_id  y hospital_death ).**")

print("\n\n**Los tipos de las columnas son:")
display( df_solution_template.dtypes )

print("\n\n**Describe:")
df_solution_template.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39308 entries, 0 to 39307
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   encounter_id    39308 non-null  int64  
 1   hospital_death  0 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 614.3 KB


None

**Los items de la solucion 2  (encounter_id  y hospital_death ).**


**Los tipos de las columnas son:


encounter_id        int64
hospital_death    float64
dtype: object



**Describe:


,encounter_id,hospital_death
count,39308.000000,0.0
mean,65346.396891,NaN
std,37920.612047,NaN
min,2.000000,NaN
25%,32568.500000,NaN
50%,65222.000000,NaN
75%,98174.500000,NaN
max,131050.000000,NaN


In [9]:
display( df_training_v2.info() )
print("**Los datos de entrenamiento tienen 186 columnas ( 170=float64, 8=int64, 8=object ).**")

print("\n\n**Los tipos de las columnas son:")
display( df_training_v2.dtypes )

print("\n\n**Describe:")
display( df_training_v2.describe() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Columns: 186 entries, encounter_id to apache_2_bodysystem
dtypes: float64(170), int64(8), object(8)
memory usage: 130.1+ MB


None

**Los datos de entrenamiento tienen 186 columnas ( 170=float64, 8=int64, 8=object ).**


**Los tipos de las columnas son:


encounter_id                     int64
patient_id                       int64
hospital_id                      int64
hospital_death                   int64
age                            float64
                                ...   
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Length: 186, dtype: object



**Describe:


,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,...,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,91713.000000,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90379.000000,91713.000000,91713.000000,91713.0,88993.000000,37334.000000,90051.000000,90612.000000,91713.000000,90998.000000,33579.000000,72451.00000,72860.000000,20845.000000,89812.000000,89812.000000,90676.000000,89812.000000,80677.000000,90835.000000,71835.000000,90998.000000,90719.000000,20845.000000,20845.000000,20845.000000,20845.000000,90479.000000,73113.000000,87605.000000,42715.000000,90998.000000,69701.000000,...,18293.000000,18293.000000,33772.000000,33772.000000,7344.000000,7344.000000,16040.000000,16040.000000,19611.000000,19611.000000,19096.000000,19096.000000,15760.000000,15760.000000,32442.000000,32442.000000,31590.000000,31590.000000,32451.000000,32451.000000,25705.000000,25705.000000,15754.000000,15754.000000,15289.000000,15289.000000,15768.000000,15768.000000,11518.000000,11518.000000,83766.000000,83766.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000
mean,65606.079280,65537.131464,105.669262,0.086302,62.309516,29.185818,0.183736,169.641588,508.357692,0.835766,0.0,84.028340,2.902968,185.401739,558.216377,0.201106,0.027979,1.147721,25.82533,1.480014,0.595751,3.465049,5.471195,0.009528,3.994778,160.326822,99.707932,32.988739,0.151223,88.015873,42.183238,42.183238,131.148467,7.353895,25.811007,137.966373,36.414472,1738.276831,0.325721,12.133333,...,33.673673,33.223156,1.596476,1.482979,3.068860,3.021543,196.096010,195.481546,4.201144,4.153006,138.239610,137.901948,13.460212,13.423475,45.248924,38.433862,7.389236,7.324530,165.913980,103.511349,285.667079,223.523037,44.668444,43.383453,7.338363,7.327883,163.841354,144.154224,244.404982,235.933050,0.086787,0.043955,0.000857,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638
std,37795.088538,37811.252183,62.854406,0.280811,16.775119,8.275142,0.387271,10.795378,228.989661,2.487756,0.0,25.011497,0.681863,86.050882,463.266985,0.400829,0.164912,2.165538,20.67298,1.525787,0.263238,0.951715,1.288376,0.097148,1.560166,90.790551,30.870502,6.873585,0.358268,42.032412,12.382412,12.382412,83.607292,0.097755,15.106312,5.279418,0.833496,1448.162411,0.468646,6.917984,...,6.840011,7.029452,0.955843,0.748557,2.926545,2.884303,92.646583,92.779494,0.763201,0.752529,5.745875,5.676796,6.979339,6.965104,14.669776,10.944916,0.084735,0.111561,108.005939,61.848052,128.218956,117.552497,14.630907,14.113107,0.105821,0.107873,113.455738,98.464543,129.964308,126.458507,0.247569,0.217341,0.029265,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169
min,1.000000,1.000000,2.000000,0.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-24.947222,0.0,38.600000,1.200

In [10]:
display( df_test_unlabeled.info() )
print("**Los datos de prueba tienen 186 columnas ( 171=float64, 7=int64, 8=object ).**")

print("\n\n**Los tipos de las columnas son:")
display( df_test_unlabeled.dtypes )

print("\n\n**Describe:")
df_test_unlabeled.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39308 entries, 0 to 39307
Columns: 186 entries, encounter_id to apache_2_bodysystem
dtypes: float64(171), int64(7), object(8)
memory usage: 55.8+ MB


None

**Los datos de prueba tienen 186 columnas ( 171=float64, 7=int64, 8=object ).**


**Los tipos de las columnas son:


encounter_id                     int64
patient_id                       int64
hospital_id                      int64
hospital_death                 float64
age                            float64
                                ...   
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Length: 186, dtype: object



**Describe:


,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,...,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,39308.000000,39308.000000,39308.000000,0.0,37654.000000,38222.000000,39308.000000,38558.000000,39308.000000,39308.000000,39308.0,38544.000000,14658.000000,38412.000000,38671.000000,39308.000000,39129.000000,14016.000000,32292.000000,32412.000000,9590.000000,38145.000000,38145.000000,38762.000000,38145.000000,34778.000000,39002.000000,31561.000000,39129.000000,39007.000000,9590.000000,9590.000000,9590.000000,9590.000000,38859.000000,32522.000000,35932.000000,24269.000000,39129.000000,30978.000000,...,9054.000000,9054.000000,15488.000000,15488.000000,4418.000000,4418.000000,8516.000000,8516.000000,9884.000000,9884.000000,9439.000000,9439.000000,8539.000000,8539.000000,13533.000000,13533.000000,14037.000000,14037.000000,13975.000000,13975.000000,11333.000000,11333.000000,6845.000000,6845.000000,7127.000000,7127.000000,7052.000000,7052.000000,5322.000000,5322.000000,34130.000000,34130.000000,39129.000000,39129.000000,39129.000000,39129.000000,39129.000000,39129.000000,39129.000000,39129.000000
mean,65346.396891,65504.743462,107.102524,NaN,61.313991,28.947566,0.199908,169.518118,1020.677267,0.829561,0.0,83.266194,2.843310,185.712329,584.282281,0.216572,0.028061,1.329411,25.476898,1.485251,0.595739,3.542928,5.516817,0.015918,4.113514,159.696647,100.197118,32.946640,0.169286,85.283750,42.115647,42.115647,134.054776,7.348349,23.614252,137.898008,36.435611,1910.924935,0.341563,12.310268,...,33.859013,33.623349,1.538802,1.422492,2.963062,2.905888,190.094880,188.894199,4.164225,4.134895,138.002755,137.819896,13.261804,13.183837,45.572279,38.776825,7.384313,7.319174,162.961737,101.687251,291.858876,224.896239,44.280409,43.248097,7.334877,7.327511,160.983778,149.841105,253.938470,247.332789,0.085675,0.039732,0.001431,0.016995,0.195661,0.015027,0.024534,0.007871,0.004319,0.021365
std,37920.612047,37878.696746,64.949586,NaN,16.828987,8.243995,0.399936,10.924032,49.232587,2.538512,0.0,24.872050,0.707919,85.406424,473.550462,0.411913,0.165149,2.743675,20.726131,1.582772,0.262257,0.908975,1.228821,0.125159,1.483078,90.479744,30.497043,6.744971,0.375009,41.558649,12.014444,12.014444,87.801566,0.099758,14.718089,5.358030,0.913549,1464.835586,0.474240,6.959027,...,6.745020,6.832450,0.924830,0.712119,2.843964,2.798316,92.115079,92.431670,0.753872,0.743224,5.638693,5.619606,6.858669,6.832913,14.522729,11.129818,0.087667,0.111557,104.978471,60.750118,134.795334,122.515109,14.645274,13.935707,0.106294,0.110464,110.703514,103.882150,134.173894,132.524123,0.255675,0.236340,0.037804,0.129254,0.396713,0.121663,0.154703,0.088372,0.065578,0.144600
min,2.000000,11.000000,1.000000,NaN,16.000000,14.844926,0.000000,137.200000,906.000000,-82.028472,0.0,38.600000,1.200000,101.000000,0.010000,0.000000,0.00000

## 3. Buscando valores nulos# 

In [11]:
# Funcion para identificar columnas que tienen valores nulos
def mostrasColumnasConNulos(df):
  for i,c in enumerate( list(df.columns) ):
    nulos = df[c].isnull().sum()
    if nulos>0 :
      print(i, c, "Cantidad Nan: "+str( nulos ), "Cantidad de Unicos: " + str(df[c].unique().size ) )

In [12]:
mostrasColumnasConNulos( df_training_v2 )
print("\nExisten mucho valores nulos")

4 age Cantidad Nan: 4228 Cantidad de Unicos: 75
5 bmi Cantidad Nan: 3429 Cantidad de Unicos: 34889
7 ethnicity Cantidad Nan: 1395 Cantidad de Unicos: 7
8 gender Cantidad Nan: 25 Cantidad de Unicos: 3
9 height Cantidad Nan: 1334 Cantidad de Unicos: 402
10 hospital_admit_source Cantidad Nan: 21409 Cantidad de Unicos: 16
11 icu_admit_source Cantidad Nan: 112 Cantidad de Unicos: 6
17 weight Cantidad Nan: 2720 Cantidad de Unicos: 3410
18 albumin_apache Cantidad Nan: 54379 Cantidad de Unicos: 36
19 apache_2_diagnosis Cantidad Nan: 1662 Cantidad de Unicos: 45
20 apache_3j_diagnosis Cantidad Nan: 1101 Cantidad de Unicos: 400
22 arf_apache Cantidad Nan: 715 Cantidad de Unicos: 3
23 bilirubin_apache Cantidad Nan: 58134 Cantidad de Unicos: 363
24 bun_apache Cantidad Nan: 19262 Cantidad de Unicos: 477
25 creatinine_apache Cantidad Nan: 18853 Cantidad de Unicos: 1128
26 fio2_apache Cantidad Nan: 70868 Cantidad de Unicos: 83
27 gcs_eyes_apache Cantidad Nan: 1901 Cantidad de Unicos: 5
28 gcs_motor_ap

In [13]:
mostrasColumnasConNulos( df_test_unlabeled )
print("\nExisten mucho valores nulos")

3 hospital_death Cantidad Nan: 39308 Cantidad de Unicos: 1
4 age Cantidad Nan: 1654 Cantidad de Unicos: 75
5 bmi Cantidad Nan: 1086 Cantidad de Unicos: 19328
7 ethnicity Cantidad Nan: 171 Cantidad de Unicos: 7
8 gender Cantidad Nan: 15 Cantidad de Unicos: 3
9 height Cantidad Nan: 750 Cantidad de Unicos: 319
10 hospital_admit_source Cantidad Nan: 11948 Cantidad de Unicos: 15
11 icu_admit_source Cantidad Nan: 115 Cantidad de Unicos: 6
17 weight Cantidad Nan: 764 Cantidad de Unicos: 1912
18 albumin_apache Cantidad Nan: 24650 Cantidad de Unicos: 37
19 apache_2_diagnosis Cantidad Nan: 896 Cantidad de Unicos: 45
20 apache_3j_diagnosis Cantidad Nan: 637 Cantidad de Unicos: 384
22 arf_apache Cantidad Nan: 179 Cantidad de Unicos: 3
23 bilirubin_apache Cantidad Nan: 25292 Cantidad de Unicos: 491
24 bun_apache Cantidad Nan: 7016 Cantidad de Unicos: 125
25 creatinine_apache Cantidad Nan: 6896 Cantidad de Unicos: 970
26 fio2_apache Cantidad Nan: 29718 Cantidad de Unicos: 60
27 gcs_eyes_apache Canti

## 4. Descartando Variables categoricas 

In [14]:
print("Seleccionando variables categoricas de los datos de entrenamiento")
categorical_df_training_v2 = df_training_v2.select_dtypes(include='object')
display( categorical_df_training_v2.head() )
categorical_df_training_v2.info()

Seleccionando variables categoricas de los datos de entrenamiento


,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
0,Caucasian,M,Floor,Floor,admit,CTICU,Sepsis,Cardiovascular
1,Caucasian,F,Floor,Floor,admit,Med-Surg ICU,Respiratory,Respiratory
2,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,Metabolic,Metabolic
3,Caucasian,F,Operating Room,Operating Room / Recovery,admit,CTICU,Cardiovascular,Cardiovascular
4,Caucasian,M,NaN,Accident & Emergency,admit,Med-Surg ICU,Trauma,Trauma


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ethnicity              90318 non-null  object
 1   gender                 91688 non-null  object
 2   hospital_admit_source  70304 non-null  object
 3   icu_admit_source       91601 non-null  object
 4   icu_stay_type          91713 non-null  object
 5   icu_type               91713 non-null  object
 6   apache_3j_bodysystem   90051 non-null  object
 7   apache_2_bodysystem    90051 non-null  object
dtypes: object(8)
memory usage: 5.6+ MB


In [15]:
print("Seleccionando variables categoricas de los datos de prueba")
categorical_df_test_unlabeled = df_test_unlabeled.select_dtypes(include='object')
display( categorical_df_test_unlabeled.head() )
categorical_df_test_unlabeled.info()

Seleccionando variables categoricas de los datos de prueba


,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
0,Caucasian,M,Emergency Department,Accident & Emergency,admit,Neuro ICU,Neurological,Neurologic
1,Caucasian,F,Operating Room,Operating Room / Recovery,admit,Med-Surg ICU,Neurological,Neurologic
2,African American,M,Operating Room,Operating Room / Recovery,admit,CCU-CTICU,Cardiovascular,Cardiovascular
3,Caucasian,M,Operating Room,Operating Room / Recovery,admit,Med-Surg ICU,Respiratory,Respiratory
4,Caucasian,M,Emergency Department,Accident & Emergency,admit,Neuro ICU,Neurological,Neurologic


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39308 entries, 0 to 39307
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ethnicity              39137 non-null  object
 1   gender                 39293 non-null  object
 2   hospital_admit_source  27360 non-null  object
 3   icu_admit_source       39193 non-null  object
 4   icu_stay_type          39308 non-null  object
 5   icu_type               39308 non-null  object
 6   apache_3j_bodysystem   38412 non-null  object
 7   apache_2_bodysystem    38412 non-null  object
dtypes: object(8)
memory usage: 2.4+ MB


## 5. Separacion de datos para entrenamiento y Prueba

In [16]:
# DATOS DE ENTRENAMIENTO
# hacemos una copia de los datos de entrenamiento
df_train = df_training_v2.copy()

# obtenemos la variable dependiente 
df_Y_train = df_train['hospital_death']

# Eliminamos aquellas caracteristicas que no son numericas o tengan valores unicos
columnasUnicosValores = ['hospital_death','encounter_id','hospital_id','patient_id','icu_id','readmission_status','ethnicity']
df_X_train = df_train.drop(columnasUnicosValores, axis=1).select_dtypes(exclude=['object'])

# Dividimos los tados en entrenamiento y pruebas
X_train, X_train_test, Y_train, Y_train_test = train_test_split(df_X_train, df_Y_train, train_size=0.8, test_size=0.2, random_state=0)

# Observamos como fueron divididos los datos
print("Cantidad de datos para entrenamiento")
display( X_train.shape )
print("Cantidad de datos para pruebas")
display( X_train_test.shape )

Cantidad de datos para entrenamiento


(73370, 172)

Cantidad de datos para pruebas


(18343, 172)

In [17]:
# DATOS DE ENTRENAMIENTO
# Usamos Imputer para el manejo de datos faltantes
simpleImputer = SimpleImputer()
imputado_X_train = pd.DataFrame(simpleImputer.fit_transform(X_train))
imputado_X_train_test = pd.DataFrame(simpleImputer.transform(X_train_test))

# Imputation removed column names; put them back
imputado_X_train.columns = X_train.columns
imputado_X_train_test.columns = X_train_test.columns

In [18]:
# DATOS DE PRUEBA
# hacemos una copia de los datos de prueba
df_test = df_test_unlabeled.copy()

# obtenemos la variable dependiente 
df_Y_test = df_test['hospital_death']

# Eliminamos aquellas caracteristicas que no son numericas o tengan valores unicos
columnasUnicosValores = ['hospital_death','encounter_id','hospital_id','patient_id','icu_id','readmission_status','ethnicity']
df_X_test = df_test.drop(columnasUnicosValores, axis=1).select_dtypes(exclude=['object'])

# datos de prueba separados
display( df_X_test.shape )
df_X_test.head()

(39308, 172)

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,...,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,56.0,21.102439,0,180.3,0.104167,68.6,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,NaN,NaN,74.0,51.0,74.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,19.500595,1,145.0,0.230556,41.0,NaN,217.0,1502.02,1,0.0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,60.0,NaN,0.0,53.0,NaN,NaN,NaN,NaN,37.0,NaN,37.0,843.4368,0.0,NaN,NaN,NaN,65.0,31.0,65.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66.0,32.518596,1,170.2,0.140972,94.2,NaN,302.0,1208.06,1,0.0,NaN,22.0,1.84,NaN,2.0,5.0,0.0,1.0,81.0,124.0,39.2,0.0,115.0,NaN,NaN,NaN,NaN,31.0,130.0,35.5,1728.0000,0.0,14.8,85.0,57.0,85.0,57.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64.0,20.798520,0,182.8,0.002083,69.5,1.9,303.0,211.03,0,0.0,0.8,24.0,0.75,0.7,4.0,6.0,0.0,1.0,122.0,111.0,32.0,1.0,48.0,36.0,36.0,142.0,7.35,30.0,133.0,37.7,1434.4128,1.0,9.7,NaN,NaN,63.0,42.0,63.0,42.0,...,32.0,32.0,1.4,1.4,NaN,NaN,248.0,248.0,4.0,4.0,133.0,133.0,9.7,9.7,36.0,36.0,7.37,7.35,142.0,57.0,202.857143,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.19,0.17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,56.0,NaN,0,NaN,0.004861,66.4,NaN,217.0,1501.01,1,0.0,NaN,11.0,0.52,NaN,1.0,4.0,0.0,1.0,192.0,94.0,32.8,0.0,41.0,NaN,NaN,NaN,NaN,5.0,134.0,36.1,NaN,1.0,14.5,84.0,24.0,100.0,24.0,100.0,24.0,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Imputation
simpleImputer = SimpleImputer()
# Usamos Imputer para el manejo de datos faltantes
imputado_X_test = pd.DataFrame(simpleImputer. fit_transform(df_X_test))

# Imputation removed column names; put them back
imputado_X_test.columns = df_X_test.columns

## 6. Implementamos modelos de Machine Learning

In [20]:
def evaluarModeloScore(model, X_train, X_test, Y_train, Y_test):
    model.fit(X_train,Y_train)    
    accuracy=model.score(X_test,Y_test)

    print("Modelo: ",model)
    print("Accuracy: ",accuracy)
    preds = model.predict(X_test)
    return model, mean_absolute_error(Y_test, preds)

In [21]:
# Modelo Arbol de Decisiones
modelDecisionTreeRegressor = DecisionTreeRegressor(random_state=1)
modelDecisionTreeRegressor, score = evaluarModeloScore( modelDecisionTreeRegressor, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)

Modelo:  DecisionTreeRegressor(random_state=1)
Accuracy:  -0.4807974913018853
MAE: 0.12260807937632884


In [22]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelDecisionTreeRegressor.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelDecisionTreeRegressor.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0
1,5,0
2,7,0
3,8,0
4,10,0


In [23]:
modelXGBRegressor = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
modelXGBRegressor, score = evaluarModeloScore( modelXGBRegressor, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)
## Mean Absolute Error: 0.11499632947390195
##-----------------------------
##X_train = imputed_X_train
##X_valid = imputed_X_valid
#
##xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
#
##xgb_model.fit(X_train, y_train, 
##             early_stopping_rounds=5, 
##             eval_set=[(X_valid, y_valid)], 
##             verbose=False)
#
##predictions = xgb_model.predict(X_valid)
#
##print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[01:24:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Modelo:  XGBRegressor(learning_rate=0.05, n_estimators=1000, n_jobs=4)
Accuracy:  0.335622635367398
MAE: 0.11489824375136366


In [24]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelXGBRegressor.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelXGBRegressor.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.005711
1,5,0.002871
2,7,-0.002865
3,8,0.147562
4,10,0.370174


In [25]:
modelSVR = SVR()
modelSVR, score = evaluarModeloScore( modelSVR, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)
##Modelo:  SVR()
##Accuracy:  0.03071240426525712
##MAE: 0.16662195943309246

Modelo:  SVR()
Accuracy:  0.03071240426525712
MAE: 0.16662195943309246


In [26]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelSVR.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelSVR.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.089299
1,5,0.090882
2,7,0.094293
3,8,0.096374
4,10,0.098226


In [27]:
modelLinearModelRidge = linear_model.Ridge(alpha=.5)
modelLinearModelRidge, score = evaluarModeloScore( modelLinearModelRidge, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)
##Modelo:  Ridge(alpha=0.5)
##Accuracy:  0.25921436267450737
##MAE: 0.13916450251823517

Modelo:  Ridge(alpha=0.5)
Accuracy:  0.25921436267450737
MAE: 0.13916450251823517


In [28]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelLinearModelRidge.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelLinearModelRidge.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.053164
1,5,-0.052564
2,7,0.053760
3,8,0.147553
4,10,0.175694


In [29]:
modelLinearModelLassoLars = linear_model.LassoLars(alpha=.1, normalize=False)
modelLinearModelLassoLars, score = evaluarModeloScore( modelLinearModelLassoLars, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)
##Modelo:  LassoLars(alpha=0.1, normalize=False)
##Accuracy:  0.12966314111211286
##MAE: 0.15174781441453442

Modelo:  LassoLars(alpha=0.1, normalize=False)
Accuracy:  0.12966314111211286
MAE: 0.15174781441453442


In [30]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelLinearModelLassoLars.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelLinearModelLassoLars.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.081594
1,5,0.028829
2,7,0.034877
3,8,0.168962
4,10,0.053014


In [31]:
modelLinearModelBR = linear_model.BayesianRidge()
modelLinearModelBR, score = evaluarModeloScore( modelLinearModelBR, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)
##Modelo:  BayesianRidge()
##Accuracy:  0.2593604329121887
##MAE: 0.13913391675326037

Modelo:  BayesianRidge()
Accuracy:  0.2593604329121887
MAE: 0.13913391675326037


In [32]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelLinearModelBR.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelLinearModelBR.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.053726
1,5,-0.053915
2,7,0.058649
3,8,0.144079
4,10,0.173317


In [33]:
modelLinearModelLR = LogisticRegression(C=0.1, penalty="l1", tol=0.01, solver="saga")
modelLinearModelLR, score = evaluarModeloScore( modelLinearModelLR, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)


Modelo:  LogisticRegression(C=0.1, penalty='l1', solver='saga', tol=0.01)
Accuracy:  0.9152810336368097
MAE: 0.08471896636319032


In [34]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelLinearModelLR.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelLinearModelLR.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0
1,5,0
2,7,0
3,8,0
4,10,0


In [35]:
modelRFR = RandomForestRegressor(max_depth=15)
modelRFR, score = evaluarModeloScore( modelRFR, imputado_X_train, imputado_X_train_test, Y_train, Y_train_test )
print('MAE:', score)


Modelo:  RandomForestRegressor(max_depth=15)
Accuracy:  0.3095748451068834
MAE: 0.11287568279351369


In [36]:
# Realizamos la prediccion para los datos de prueba
prediccion = modelRFR.predict(imputado_X_test)

# Guardamos la prediccion de los datos
resultado= pd.DataFrame({'encounter_id': df_test_unlabeled.encounter_id,
                       'hospital_death': prediccion},dtype=np.int32)
 
resultado.to_csv('modelRFR.csv', index=False)
resultado.head()

,encounter_id,hospital_death
0,2,0.052399
1,5,0.029435
2,7,0.006808
3,8,0.100137
4,10,0.373645
